In [1]:
import os
import sys
import pprint

base_dir = 'F:\Thesis_ssd\MasterThesis3.0'
os.chdir(base_dir)

from Classes.DataProcessing.LoadData import LoadData
from Classes.DataProcessing.DataHandler import DataHandler
from Classes.DataProcessing.DataGenerator import DataGenerator
from Classes.Modeling.GridSearchResultProcessor import GridSearchResultProcessor
from Classes.Modeling.CustomCallback import CustomCallback
from Classes.Scaling.ScalerFitter import ScalerFitter
from Classes.Scaling.MinMaxScalerFitter import MinMaxScalerFitter
from Classes.Scaling.StandardScalerFitter import StandardScalerFitter
from Classes.Modeling.ResultFitter import ResultFitter
import json

In [16]:
load_args = {
    'earth_explo_only' : True,
    'noise_earth_only' : False,
    'downsample' : True,
    'upsample' : True,
    'frac_diff' : 0.5,
    'seed' : 1,
    'subsample_size' : 0.25
}

loadData = LoadData(**load_args)

full_ds, train_ds, val_ds, test_ds = loadData.get_datasets()
noise_ds = loadData.noise_ds
handler = DataHandler(loadData)
dataGen = DataGenerator(loadData)
resultFitter = ResultFitter(loadData)

AttributeError: 'LoadData' object has no attribute 'getDatasets'

In [6]:
result_file_name = 'results_6_noiseNotNoise_timeAug_sscale_noiseAug_earlyS.csv'
model = resultFitter.fit_from_csv_and_index(result_file_name, 0, 2, 10, use_tensorboard = False, 
                               use_liveplots = True, use_custom_callback = False)

NameError: name 'resultFitter' is not defined

In [39]:
result_file_name = 'results_LSTM_NARROW_noiseNotNoise_timeAug_sscale_noiseAug_earlyS.csv'
df = GridSearchResultProcessor().get_results_df_by_name(result_file_name, 2)

In [40]:
df[df.columns[:15]]

,batch_size,epochs,learning_rate,num_layers,optimizer,activation,decay_sequence,dropout_rate,filters,kernel_size,l1_r,l2_r,output_layer_activation,padding,start_neurons
0,64,33,0.01,2,sgd,tanh,"[1, 2]",0.500,17,5,0.0001,0.001,sigmoid,same,32
1,64,33,0.01,2,sgd,tanh,"[1, 2]",0.500,17,5,0.0001,0.001,sigmoid,same,32
2,64,33,0.01,2,sgd,tanh,"[1, 2]",0.400,17,5,0.0001,0.001,sigmoid,same,32
3,64,33,0.01,2,sgd,tanh,"[1, 2]",0.200,17,5,0.0001,0.001,sigmoid,same,32
4,64,33,0.01,2,sgd,tanh,"[1, 2]",0.100,17,5,0.0001,0.001,sigmoid,same,32
5,64,33,0.01,2,sgd,tanh,"[1, 2]",0.010,17,5,0.0001,0.001,sigmoid,same,32
6,64,33,0.01,2,sgd,tanh,"[1, 2]",0.001,17,5,0.0001,0.001,sigmoid,same,32
7,64,33,0.01,2,sgd,tanh,"[1, 2]",0.000,17,5,0.0001,0.001,sigmoid,same,32
8,64,33,0.01,2,adam,tanh,"[1, 2]",0.300,17,5,0.0001,0.001,sigmoid,same,32
9,64,33,0.01,2,rmsprop,tanh,"[1, 2]",0.300,17,5,0.0001,0.001,sigmoid,same,32


In [41]:
df[df.columns[13:]]

,padding,start_neurons,train_loss,train_accuracy,train_precision,train_recall,val_loss,val_accuracy,val_precision,val_recall
0,same,32,0.843425,0.928650,0.862717,0.857514,0.970667,0.868316,0.775568,0.778812
1,same,32,0.856955,0.949212,0.845839,0.809346,0.980064,0.903571,0.766572,0.724409
2,same,32,0.832348,0.944839,0.879387,0.867630,1.003364,0.888981,0.783144,0.768074
3,same,32,0.759288,0.980216,0.920956,0.898200,1.083491,0.900166,0.794034,0.774517
4,same,32,0.765186,0.966016,0.935662,0.935273,1.217005,0.874209,0.786458,0.790981
5,same,32,0.782132,0.983592,0.944435,0.931207,1.338903,0.889803,0.787405,0.774517
6,same,32,0.793265,0.982209,0.929980,0.910209,1.372538,0.886949,0.779356,0.763780
7,same,32,1.044652,0.971432,0.922752,0.909552,1.513449,0.885833,0.776989,0.760916
8,same,32,2.342889,0.822314,0.720338,0.734196,2.349560,0.814815,0.708807,0.724409
9,same,32,1.584430,0.763097,0.711021,0.813967,1.595288,0.757818,0.705492,0.815319


## Findings:
 - New optimal dropout_rate: 0.2 (train_acc: 98%, val_acc: 90%) Old 0.3 (train_acc: 96.8%, val_acc: 90.9%)
 - RELU and Sigmoid activation functions do not work with this model.
 - L1 regularization is counter productive (Less is more) so let l1 = 0.00
 - 32 start neurons appear to be optimal

In [ ]:
li = fit_from_csv_and_index(result_file_name, 0, 2)

In [ ]:
mysplit(result_fi)

In [60]:
class A(B):
    def __init__(self, hey):
        self.hey = hey

class B:
    def __init__(self):
        pass
    def check(self):
        print(hasattr(self, 'hey'))

In [61]:
A = A("bye")

In [62]:
A.check()

AttributeError: 'A' object has no attribute 'check'

In [58]:
hasattr(A, 'hey')

True